*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
# !pip install tensorflow
# !pip install numpy --upgrade --ignore-installed
!pip install -q tensorflow-text

import requests
import tensorflow_text as tf_text
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_file_path = "https://cdn.freecodecamp.org/project-data/sms/train-data.tsv"
test_file_path = "https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv"

In [ ]:
# train_files = pd.read_csv('https://cdn.freecodecamp.org/project-data/sms/train-data.tsv', header=0, names=['label', 'message'], sep='\t')
train_files = pd.read_csv('train-data.tsv', sep='\t', header=None, names=['Classification', 'Message'])
valid_files = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=['Classification', 'Message'])

In [ ]:
# Configure parameters
batch_size = 32
epochs = 15
BUFFER_SIZE = 10000


In [ ]:
!mkdir trainData
!mv train-data.tsv trainData
!mkdir valData
!mv valid-data.tsv valData

In [ ]:
# this shows there are duplicates that probably need to be removed from the datasets
train_files.describe()
duplicatedRow = train_files[train_files.duplicated()]
print(duplicatedRow[:10])

In [ ]:
# Remove duplicate entries
train_files = train_files.drop_duplicates()
valid_files = valid_files.drop_duplicates()
train_files.describe()

In [ ]:
train_files['Classification'].value_counts()
valid_files['Classification'].value_counts()

# ham messages are much more frequent than spam, so we should balance the data before training the model with it
lenTrainSpam = len(train_files[train_files['Classification'] == 'spam'])

# gather lenTrainSpam amount of random samples where message is considered ham
trainHam = train_files[train_files['Classification'] == 'ham'].sample(n=lenTrainSpam)
trainSpam = train_files[train_files['Classification'] == 'spam'].sample(n=lenTrainSpam)

# Merge lenTrainSpam amount of ham and spam samples into one dataframe
trainSet = trainHam.merge(trainSpam,how='outer', on=None)

# Shuffle dataset
trainSet = trainSet.sample(len(trainSet), random_state=1)
trainSet

In [ ]:
# Encode hams as 0, spams as 1 to allow for a binary model
trainSet['Classification'] = trainSet['Classification'].map({'ham':0, 'spam':1})
valid_files['Classification'] = valid_files['Classification'].map({'ham':0, 'spam':1})

trainSet.head()

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token=oov_tok)
tokenizer.fit_on_texts(trainSet['Message'])
X_train = tokenizer.texts_to_sequences(trainSet['Message'])
X_test = tokenizer.texts_to_sequences(valid_files['Message'])

X_train_padded = pad_sequences(X_train, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

Y_test = valid_files['Classification']
Y_train = trainSet['Classification']

Y_train_array = np.array(Y_train)
Y_test_array = np.array(Y_test)

word_index = tokenizer.word_index

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
  
    tf.keras.layers.Dense(8, activation='relu'),  tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 55
history = model.fit(X_train_padded, Y_train_array, epochs=num_epochs, validation_data=(X_test_padded, Y_test_array))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  predictions = []
  pred_text = pd.Series(pred_text)
  tokenized_string = tokenizer.texts_to_sequences(pred_text)
  pad_text_sequence = pad_sequences(tokenized_string, maxlen=max_length)
  result = model.predict(pad_text_sequence)[0]
  
  if result[0] <= 0.5:
    predictions = [result[0], "ham"]
  else:
    predictions = [result[0], "spam"]
  print(predictions)
  return predictions


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
